# Quantium Retail Analytics Project

![Quantium Logo](https://media.licdn.com/dms/image/v2/C4E0BAQE8LsQ-XvPyxw/company-logo_200_200/company-logo_200_200/0/1660868732692/quantium_logo?e=2147483647&v=beta&t=UX6w4Ad7XXxwfyLRaaI8eUMdmWa7fsYBSP1w602D27c)

## Chip Category Customer Analysis


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Import all libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## 2. Import the data

In [ ]:
transaction_data = pd.read_csv("/content/drive/MyDrive/Data Analyst Thực Chiến/QVI_transaction_data - in.csv")
behaviour_data = pd.read_csv("/content/drive/MyDrive/Data Analyst Thực Chiến/QVI_purchase_behaviour - QVI_purchase_behaviour.csv")

## 3. Data cleaning and preparation


> Before conducting any analysis, we need to ensure our data is clean, relevant, and properly formatted. This guide outlines the essential steps for preparing retail transaction data, focusing on techniques relevant to product sales analysis.

Hints:
1. Handle Date Formats:
We want to verify if any dates are missing within the range of the dataset. If any specfic days are missing, what can we do?

2. Filter Relevant Products: We want to ensure that we are only examining chip purchases. Non-chip products such as salsa that should be excluded.

  *Note:* You should look at PROD_NAME column and come up with your own way to filter out the non-chip products. No right or wrong answer, I look forward to seeing your problem solving skill here!!!


3. Identify and Handle Outliers: We want to ensure that we exclude all outliers (especially those in numerical columns before any further analysis)

4. Feature Engineering: Create more valuable columns for future analysis.

  First,  we truly care about is the package size (e.g. 80g, 370g). Look at the PROD_NAME again and see if you can find this data, if yes, extract them to a new column.

  Second, what about Brand name? Can we get the Brand name of all the products?

  Third, what about price per unit?

5. Data integration: Merge the transaction data with the customer data on loyalty card number?

6. Final Data Validation: Final check for missing values, illogical values, inconsistent, duplicate records.

# Transaction Data


In [ ]:
transaction_data.info() #no missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264836 entries, 0 to 264835
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DATE            264836 non-null  int64  
 1   STORE_NBR       264836 non-null  int64  
 2   LYLTY_CARD_NBR  264836 non-null  int64  
 3   TXN_ID          264836 non-null  int64  
 4   PROD_NBR        264836 non-null  int64  
 5   PROD_NAME       264836 non-null  object 
 6   PROD_QTY        264836 non-null  int64  
 7   TOT_SALES       264836 non-null  float64
dtypes: float64(1), int64(6), object(1)
memory usage: 16.2+ MB


In [ ]:
transaction_data['PROD_NAME'].unique()

array(['Natural Chip        Compny SeaSalt175g',
       'CCs Nacho Cheese    175g',
       'Smiths Crinkle Cut  Chips Chicken 170g',
       'Smiths Chip Thinly  S/Cream&Onion 175g',
       'Kettle Tortilla ChpsHny&Jlpno Chili 150g',
       'Old El Paso Salsa   Dip Tomato Mild 300g',
       'Smiths Crinkle Chips Salt & Vinegar 330g',
       'Grain Waves         Sweet Chilli 210g',
       'Doritos Corn Chip Mexican Jalapeno 150g',
       'Grain Waves Sour    Cream&Chives 210G',
       'Kettle Sensations   Siracha Lime 150g',
       'Twisties Cheese     270g', 'WW Crinkle Cut      Chicken 175g',
       'Thins Chips Light&  Tangy 175g', 'CCs Original 175g',
       'Burger Rings 220g', 'NCC Sour Cream &    Garden Chives 175g',
       'Doritos Corn Chip Southern Chicken 150g',
       'Cheezels Cheese Box 125g', 'Smiths Crinkle      Original 330g',
       'Infzns Crn Crnchers Tangy Gcamole 110g',
       'Kettle Sea Salt     And Vinegar 175g',
       'Smiths Chip Thinly  Cut Original 175g', 'K

In [ ]:
exclude_keywords = ['salsa', 'dip', 'crackers']
excluded_rows_count = transaction_data[transaction_data['PROD_NAME'].str.lower().str.contains('|'.join(exclude_keywords))].shape[0]

print(f"Number of rows containing 'salsa', 'dip', or 'crackers' before removal: {excluded_rows_count}")

Number of rows containing 'salsa', 'dip', or 'crackers' before removal: 22706


In [ ]:
exclude_keywords = ['salsa', 'dip', 'crackers']
transaction_data = transaction_data[~transaction_data['PROD_NAME'].str.lower().str.contains('|'.join(exclude_keywords))]
transaction_data['PROD_NAME'].unique()

array(['Natural Chip        Compny SeaSalt175g',
       'CCs Nacho Cheese    175g',
       'Smiths Crinkle Cut  Chips Chicken 170g',
       'Smiths Chip Thinly  S/Cream&Onion 175g',
       'Kettle Tortilla ChpsHny&Jlpno Chili 150g',
       'Smiths Crinkle Chips Salt & Vinegar 330g',
       'Grain Waves         Sweet Chilli 210g',
       'Doritos Corn Chip Mexican Jalapeno 150g',
       'Grain Waves Sour    Cream&Chives 210G',
       'Kettle Sensations   Siracha Lime 150g',
       'Twisties Cheese     270g', 'WW Crinkle Cut      Chicken 175g',
       'Thins Chips Light&  Tangy 175g', 'CCs Original 175g',
       'Burger Rings 220g', 'NCC Sour Cream &    Garden Chives 175g',
       'Doritos Corn Chip Southern Chicken 150g',
       'Cheezels Cheese Box 125g', 'Smiths Crinkle      Original 330g',
       'Infzns Crn Crnchers Tangy Gcamole 110g',
       'Kettle Sea Salt     And Vinegar 175g',
       'Smiths Chip Thinly  Cut Original 175g', 'Kettle Original 175g',
       'Red Rock Deli Thai  C

In [ ]:
transaction_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 242130 entries, 0 to 264835
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DATE            242130 non-null  int64  
 1   STORE_NBR       242130 non-null  int64  
 2   LYLTY_CARD_NBR  242130 non-null  int64  
 3   TXN_ID          242130 non-null  int64  
 4   PROD_NBR        242130 non-null  int64  
 5   PROD_NAME       242130 non-null  object 
 6   PROD_QTY        242130 non-null  int64  
 7   TOT_SALES       242130 non-null  float64
dtypes: float64(1), int64(6), object(1)
memory usage: 16.6+ MB


In [ ]:
transaction_data.describe()
transaction_data['PROD_QTY'].value_counts() # 2 for 200 -> exclude
transaction_data = transaction_data[transaction_data['PROD_QTY'] != 200]
transaction_data['TOT_SALES'].value_counts() # 2 for 650 -> exclude
transaction_data = transaction_data[transaction_data['TOT_SALES'] != 650]

In [ ]:
transaction_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 242128 entries, 0 to 264835
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DATE            242128 non-null  int64  
 1   STORE_NBR       242128 non-null  int64  
 2   LYLTY_CARD_NBR  242128 non-null  int64  
 3   TXN_ID          242128 non-null  int64  
 4   PROD_NBR        242128 non-null  int64  
 5   PROD_NAME       242128 non-null  object 
 6   PROD_QTY        242128 non-null  int64  
 7   TOT_SALES       242128 non-null  float64
dtypes: float64(1), int64(6), object(1)
memory usage: 16.6+ MB


In [ ]:
transaction_data.describe()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_QTY,TOT_SALES
count,242128.000000,242128.000000,2.421280e+05,2.421280e+05,242128.000000,242128.000000,242128.000000
mean,43464.096156,135.069137,1.355431e+05,1.351498e+05,55.649661,1.906450,7.330916
std,105.397481,76.788658,8.065247e+04,7.815246e+04,33.599908,0.342277,2.485716
min,43282.000000,1.000000,1.000000e+03,1.000000e+00,1.000000,1.000000,1.700000
25%,43373.000000,70.000000,7.001500e+04,6.756775e+04,26.000000,2.000000,5.800000
50%,43464.000000,130.000000,1.303675e+05,1.351880e+05,52.000000,2.000000,7.400000
75%,43555.000000,203.000000,2.030910e+05,2.026872e+05,85.000000,2.000000,8.800000
max,43646.000000,272.000000,2.373711e+06,2.415841e+06,114.000000,5.000000,29.500000


In [ ]:
# Extract package size and clean product name
transaction_data['PACKAGE_SIZE'] = transaction_data['PROD_NAME'].str.extract(r'(\d+\s*[gG])')[0].str.lower().str.replace(' ', '') #use REGEX
transaction_data['CLEAN_PROD_NAME'] = transaction_data['PROD_NAME'].str.replace(r'\d+\s*[gG]', '', regex=True).str.strip()
transaction_data

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACKAGE_SIZE,CLEAN_PROD_NAME
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175g,Natural Chip Compny SeaSalt
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3,175g,CCs Nacho Cheese
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9,170g,Smiths Crinkle Cut Chips Chicken
3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0,175g,Smiths Chip Thinly S/Cream&Onion
4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8,150g,Kettle Tortilla ChpsHny&Jlpno Chili
...,...,...,...,...,...,...,...,...,...,...
264831,43533,272,272319,270088,89,Kettle Sweet Chilli And Sour Cream 175g,2,10.8,175g,Kettle Sweet Chilli And Sour Cream
264832,43325,272,272358,270154,74,Tostitos Splash Of Lime 175g,1,4.4,175g,Tostitos Splash Of Lime
264833,43410,272,272379,270187,51,Doritos Mexicana 170g,2,8.8,170g,Doritos Mexicana
264834,43461,272,272379,270188,42,Doritos Corn Chip Mexican Jalapeno 150g,2,7.8,150g,Doritos Corn Chip Mexican Jalapeno


In [ ]:
transaction_data['BRAND'] = transaction_data['CLEAN_PROD_NAME'].str.split().str[0]
transaction_data['BRAND'].value_counts()

,count
BRAND,
Kettle,41288
Smiths,25952
Pringles,25102
Doritos,22041
Thins,14075
RRD,11894
WW,10320
Cobs,9693
Tostitos,9471


In [ ]:
brand_corrections = {
    'Smith': 'Smiths',
    'Dorito': 'Doritos',
    'Infzns': 'Infuzions',
    'GrnWves': 'Grain Waves',
    'Grain': 'Grain Waves',
    'Red': 'Red Rock Deli',
    'RRD': 'Red Rock Deli',
    'WW': 'Woolworths',
    'Natural': 'Natural Chip Co.',
    'Snbts': 'Sunbites',
    'Burger': 'Burger Rings',
    'CCs': 'CCs',
    'NCC': 'Natural Chip Co.',
    'French': 'French Fries'
}
transaction_data['BRAND'] = transaction_data['BRAND'].replace(brand_corrections)
transaction_data['BRAND'].value_counts()


,count
BRAND,
Kettle,41288
Smiths,28915
Doritos,25224
Pringles,25102
Red Rock Deli,16321
Thins,14075
Woolworths,11836
Infuzions,11027
Cobs,9693


In [ ]:
transaction_data['PRICE_PER_UNIT'] = transaction_data['TOT_SALES'] / transaction_data['PROD_QTY']

In [ ]:
data = pd.merge(transaction_data, behaviour_data, on = 'LYLTY_CARD_NBR', how = 'left')
data

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACKAGE_SIZE,CLEAN_PROD_NAME,BRAND,PRICE_PER_UNIT,LIFESTAGE,PREMIUM_CUSTOMER
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175g,Natural Chip Compny SeaSalt,Natural Chip Co.,3.00,YOUNG SINGLES/COUPLES,Premium
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3,175g,CCs Nacho Cheese,CCs,2.10,MIDAGE SINGLES/COUPLES,Budget
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9,170g,Smiths Crinkle Cut Chips Chicken,Smiths,1.45,MIDAGE SINGLES/COUPLES,Budget
3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0,175g,Smiths Chip Thinly S/Cream&Onion,Smiths,3.00,MIDAGE SINGLES/COUPLES,Budget
4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8,150g,Kettle Tortilla ChpsHny&Jlpno Chili,Kettle,4.60,MIDAGE SINGLES/COUPLES,Budget
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242123,43533,272,272319,270088,89,Kettle Sweet Chilli And Sour Cream 175g,2,10.8,175g,Kettle Sweet Chilli And Sour Cream,Kettle,5.40,YOUNG SINGLES/COUPLES,Premium
242124,43325,272,272358,270154,74,Tostitos Splash Of Lime 175g,1,4.4,175g,Tostitos Splash Of Lime,Tostitos,4.40,YOUNG SINGLES/COUPLES,Premium
242125,43410,272,272379,270187,51,Doritos Mexicana 170g,2,8.8,170g,Doritos Mexicana,Doritos,4.40,YOUNG SINGLES/COUPLES,Premium
242126,43461,272,272379,270188,42,Doritos Corn Chip Mexican Jalapeno 150g,2,7.8,150g,Doritos Corn Chip Mexican Jalapeno,Doritos,3.90,YOUNG SINGLES/COUPLES,Premium


In [ ]:
data['LYLTY_CARD_NBR'].nunique()

70925

## 4. Exploratory Data Analysis


> After data cleaning, exploratory data analysis helps us understand patterns, discover insights, and identify areas for deeper investigation. This guide outlines key EDA steps for retail transaction data.




In [ ]:
data.groupby('BRAND')['TOT_SALES'].sum().sort_values(ascending = False)

,TOT_SALES
BRAND,
Kettle,390239.8
Doritos,226329.9
Smiths,210446.0
Pringles,177655.5
Thins,88852.5
Red Rock Deli,87607.5
Twisties,81522.1
Tostitos,79789.6
Infuzions,75936.0


In [ ]:
data.groupby('PACKAGE_SIZE')[['TOT_SALES', 'PROD_QTY']].sum().sort_values(ascending = False, by = 'TOT_SALES')

,TOT_SALES,PROD_QTY
PACKAGE_SIZE,,
175g,485437.4,126467
150g,282635.8,73952
134g,177655.5,48019
170g,146673.0,38088
110g,139653.8,36751
330g,136794.3,23999
165g,101360.6,29051
380g,75419.6,12273
270g,55425.4,12049


In [ ]:
data.groupby('PRICE_PER_UNIT')['PROD_QTY'].sum()

,PROD_QTY
PRICE_PER_UNIT,
1.32,5
1.35,4
1.44,5
1.45,2
1.48,5
1.65,2
1.70,11124
1.80,2872
1.84,5


In [ ]:
data.groupby('LIFESTAGE')['TOT_SALES'].sum().sort_values(ascending = False)

,TOT_SALES
LIFESTAGE,
OLDER SINGLES/COUPLES,369736.45
RETIREES,336766.30
OLDER FAMILIES,323092.70
YOUNG FAMILIES,289572.70
YOUNG SINGLES/COUPLES,239568.00
MIDAGE SINGLES/COUPLES,169712.40
NEW FAMILIES,46571.55


In [ ]:
data.groupby('PREMIUM_CUSTOMER')['TOT_SALES'].sum().sort_values(ascending = False)

,TOT_SALES
PREMIUM_CUSTOMER,
Mainstream,689264.80
Budget,620512.65
Premium,465242.65


In [ ]:
data.to_csv("/content/drive/MyDrive/Data Analyst Thực Chiến/cleaned_data.csv", index=False)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242128 entries, 0 to 242127
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   DATE              242128 non-null  int64  
 1   STORE_NBR         242128 non-null  int64  
 2   LYLTY_CARD_NBR    242128 non-null  int64  
 3   TXN_ID            242128 non-null  int64  
 4   PROD_NBR          242128 non-null  int64  
 5   PROD_NAME         242128 non-null  object 
 6   PROD_QTY          242128 non-null  int64  
 7   TOT_SALES         242128 non-null  float64
 8   PACKAGE_SIZE      242128 non-null  object 
 9   CLEAN_PROD_NAME   242128 non-null  object 
 10  BRAND             242128 non-null  object 
 11  PRICE_PER_UNIT    242128 non-null  float64
 12  LIFESTAGE         242128 non-null  object 
 13  PREMIUM_CUSTOMER  242128 non-null  object 
dtypes: float64(2), int64(6), object(6)
memory usage: 25.9+ MB
